<a href="https://colab.research.google.com/github/KyriakiGeorgiou/KyriakiGeorgiou/blob/main/Job_Description_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%capture
!pip install -U https://gradio-builds.s3.amazonaws.com/a4559af9e1bc6226fc0130d9ab32cc161e99efdd/gradio-3.36.1-py3-none-any.whl

In [3]:
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS

import gradio as gr
from gradio.inputs import Textbox, File

In [4]:
%%capture
!pip install transformers
!pip install spacy
!pip install PyMuPDF
!pip install PyPDF2
!pip install pypdf
!pip install spacy_transformers
!pip install nltk
!pip install -U sentence-transformers


In [5]:
import os
import scipy
import numpy as np
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from xml.etree import ElementTree as ET


import spacy
from spacy.tokens import DocBin
from spacy.tokens import Doc, Span

import string
import re
import torch

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize


from sentence_transformers import SentenceTransformer, util

import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS


from collections import Counter
import sys, fitz

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
def read_resume(resume_file):

  fname = resume_file
  doc = fitz.open(fname)

  text = " "
  page_count = 0
  for page in doc:
    text = text + str(page.get_text())

  return text

def clean_CV_simple(resume):

    cleaned_text = resume.strip()
    cleaned_text = cleaned_text.replace("•", "")
    cleaned_text = cleaned_text.replace("_", "")
    cleaned_text = cleaned_text.replace("➢", "")
    cleaned_text = cleaned_text.replace(",","").replace("(","").replace(".","").replace(")","").replace("‘","").replace(":","").replace("&","").replace(";","").replace("/","")
    cleaned_text = cleaned_text.replace("etc.","")
    cleaned_text = re.sub(r'\band\b', '', cleaned_text, flags=re.IGNORECASE) # Remove standalone "and" while preserving it if it is part of a word

    stop_words = set(stopwords.words('english'))
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word.lower() not in stop_words]) # Remove stopwords
    cleaned_text = " ".join(cleaned_text.split())  # Remove unnecessary spaces

    return cleaned_text


def clean_description_func(desc):

    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)

    cleaned_text = desc.strip()
    cleaned_text = emoji_pattern.sub(r'', cleaned_text)
    cleaned_text = cleaned_text.replace(",","").replace(")","").replace("(","").replace("etc.","").replace("-","").replace(".","").replace(":","").replace("/","").replace("’","")

    cleaned_text = " ".join(cleaned_text.split())  # Remove unnecessary spaces

    stop_words = set(stopwords.words('english'))
    cleaned_text = ' '.join([word for word in cleaned_text.split() if word.lower() not in stop_words])   # Remove stopwords

    return cleaned_text

def generate_word_cloud(clean_description):
  word_cloud = WordCloud(
    width=3000,
    height=2000,
    random_state=1,
    background_color="white",
    colormap="viridis",
    collocations=False,
    stopwords=STOPWORDS,
    ).generate(clean_description)

    # Display the generated Word Cloud
  plt.imshow(word_cloud)
  plt.axis("off")

  plt.show()

  return plt

# Load a pre-trained SBERT model
sentence_model = SentenceTransformer('stsb-roberta-base')
#'stsb-roberta-base'
#all-distilroberta-v1
# Load NER model
nlp = spacy.load('/content/drive/MyDrive/xml_model')

# Compute similarity scores
def compute_similarity(description, resume):
    scores = []
    average_scores = []  # List to store average scores

    for desc_sent in description:
        desc_embedding = sentence_model.encode([desc_sent], convert_to_tensor=True)
        desc_scores = []
        for resume_sent in resume:
            resume_embedding = sentence_model.encode([resume_sent], convert_to_tensor=True)
            score = util.cos_sim(desc_embedding, resume_embedding)
            if score <= 0.10:
              desc_scores.append(score.item()-0.20)
            elif score <= 0.15:
              desc_scores.append(score.item()-0.15)
            elif score >= 0.70:
              desc_scores.append(score.item()+0.60)
            elif score >= 0.50:
              desc_scores.append(score.item()+0.40)
            elif score >= 0.30:
              desc_scores.append(score.item()+0.30)
            elif score >=0.20:
              desc_scores.append(score.item()+0.10)

            else:
              desc_scores.append(score.item())
        if len(desc_scores) > 0:
            average_score = sum(desc_scores) / len(desc_scores)
        else:
            average_score = 0.0  # or any other value you prefer when desc_scores is empty
        average_scores.append(average_score)
        # print("Average Similarity Score:", average_score)

    return average_scores

def estimate_overall_similarity(scores):
    # Calculate the average similarity score
    total_scores = sum(scores)
    print("Total scores", total_scores)
    num_scores = len(scores)
    print("Num scores", num_scores)

    if num_scores > 0:
        average_score = total_scores / num_scores
    else:
        average_score = 0.0  # or any other value you prefer when num_scores is zero

    return average_score

def output_score(description,resume):

  # read resume
  resume_text = read_resume(resume)
  # clean description
  clean_description = clean_description_func(description)
  # clean resume
  clean_resume = clean_CV_simple(resume_text)

  # Load NER model

  doc = nlp(clean_resume)

  # Extract entities from resume
  tech_tools = []
  soft_skills = []
  job_specific_skills = []
  degree = []
  years_experience = []
  companies_worked = []
  designation = []

  for ent in doc.ents:
    if ent.label_ == "Tech Tools":
      tech_tools.append(ent.text)
    elif ent.label_ == "Soft Skills":
      soft_skills.append(ent.text)
    elif ent.label_ == "Job Specific Skills":
      job_specific_skills.append(ent.text)
    elif ent.label_ == "Degree":
      degree.append(ent.text)
    elif ent.label_ == "Years of Experience":
      years_experience.append(ent.text)
    elif ent.label_ == "Companies worked at":
      companies_worked.append(ent.text)
    elif ent.label_ == "Designation":
      designation.append(ent.text)

  doc_desc = nlp(clean_description)

  tech_tools_desc = []
  soft_skills_desc = []
  job_specific_tools_desc = []
  degree_desc = []
  years_desc = []
  companies_worked_desc = []
  designation_desc = []

  for ent in doc_desc.ents:
    if ent.label_ == "Tech Tools":
      tech_tools_desc.append(ent.text)
    elif ent.label_ == "Soft Skills":
      soft_skills_desc.append(ent.text)
    elif ent.label_ == "Job Specific Skills":
      job_specific_tools_desc.append(ent.text)
    elif ent.label_ == "Degree":
      degree_desc.append(ent.text)
    elif ent.label_ == "Years of Experience":
      years_desc.append(ent.text)
    elif ent.label_ == "Companies worked at":
      companies_worked_desc.append(ent.text)
    elif ent.label_ == "Designation":
      designation_desc.append(ent.text)

  desc_list = [tech_tools_desc, soft_skills_desc, job_specific_tools_desc, degree_desc, companies_worked_desc, designation_desc, years_desc]
  resume_list = [tech_tools, soft_skills, job_specific_skills, degree, companies_worked, designation, years_experience]

  resume_skills = tech_tools[:2] + soft_skills[:2] + job_specific_skills[:2]
  desc_skills = tech_tools_desc[:2] + soft_skills_desc[:2] + job_specific_tools_desc[:2]

  rest_desc = degree_desc[:2] + companies_worked_desc[:2] + designation_desc[:2] + years_desc[:2]
  rest_resume = degree[:2] + companies_worked[:2] + designation[:2] + years_experience[:2]

  individual_score = (compute_similarity(desc_skills[:50], resume_skills[:50]))
  individual_score_2 = (compute_similarity(rest_desc[:40], rest_resume[:40]))

  overall_score = estimate_overall_similarity(individual_score) + estimate_overall_similarity(individual_score_2)
  print("Overall Similarity Score:", overall_score)

  word_cloud_desc = """To help you better visualize the skills the job requires, here's a word cloud highlighting the key skills from the job
      description. The bigger the word, the more frequently it appeared on the job description (and the more important it likely is)."""


  return overall_score, word_cloud_desc , generate_word_cloud("".join(tech_tools+job_specific_skills))

inputs = [
    Textbox(label="Enter description", lines=5),
    File(label="Upload your CV in PDF format")
]

outputs = [
    gr.Label(label = "Suitability Score"),
    gr.Label(label="Description"),
    gr.Plot( label="Word Cloud")
]

interface = gr.Interface(
    fn=output_score,
    inputs=inputs,
    outputs=outputs
)

interface.launch(share = False,debug = True)

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_pipeline' (0.0.0) was trained with spaCy v3.5.2 and may not be 100% compatible with the current version (3.6.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
<ipython-input-6-1ba762e4a7b6>:225: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  Textbox(label="Enter description", lines=5),
<ipython-input-6-1ba762e4a7b6>:225: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  Textbox(label="Enter description", lines=5),
<ipython-input-6-1ba762e4a7b6>:225: GradioDeprecationWarning: `numeric` parameter is deprecated, and it has no effect
  Textbox(label="Enter description", lin

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>